In [1]:
import imageio
import numpy as np
from scipy.interpolate import RectBivariateSpline, RegularGridInterpolator
from scipy.ndimage.morphology import distance_transform_edt

def assemble_image_weights( K ):
    """Return weight matrix W used for image merging.

    Notes:
        - Calculates Euclidean distance for each entry to nearest non-zero pixel value
        - edt: Euclidean Distance Transform

    Arguments:
        K (np.ndarray): Pixel intensity

    Returns:
        W (np.ndarray): Pixel weights used for merging images
    """
    # NaN in K indicates no pixel value at that location
    # edt finds euclidean distance from no value to closest value
    # so, find the nans, then invert so it works with the function
    W = distance_transform_edt(~np.isnan(K[:, :, 0]))

    # Not sure when this would happen, but I'm including it because it's in the MATLAB code
    if np.isinf(np.max(W)):
        W[:] = 1

    W = W / np.max(W)
    #W[W == 0] = np.nan

    return W

def apply_weights_to_pixels( K, W ):
    """Return pixel intensities (K) weighted by W.

    Arguments:
        K (np.ndarray): Pixel intensity
        W (np.ndarray): Pixel weights used for merging images

    Returns:
        K_weighted(np.ndarray): Pixel intensity weighted for merging
    """
    W_nonan = W.copy()
    #W_nonan[np.isnan(W_nonan)] = 0
    K_weighted = K*W_nonan[:, :, np.newaxis]

    return K_weighted


In [4]:

K = np.zeros((5,5))

K[1,4]=1
K[2,3:]=1
K[3,2:]=1
K[4,1:]=1
Kp = np.tile(K[:,:,None],[1,1,3])
print('Kp')
print(np.shape(Kp))
print(Kp[:,:,0])

W = assemble_image_weights( Kp )
print('W')
print(W[:,:])

Kw = apply_weights_to_pixels( Kp, W )
print('Kw')
print(Kw[:,:,0])

# add up weights and pixel itensities
totalW = np.zeros_like(Kp)
M = np.zeros_like(Kp)
totalW = totalW + W[:, :, np.newaxis]
#K_weighted[np.isnan(K_weighted)] = 0
M = M + Kw

# stop divide by 0 warnings
with np.errstate(invalid='ignore'):
    M = M / totalW
print(M[:,:,0])

Kp
(5, 5, 3)
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]
W
[[0.15617376 0.22086305 0.34921515 0.4938648  0.64392092]
 [0.31234752 0.34921515 0.4417261  0.56309251 0.6984303 ]
 [0.46852129 0.4938648  0.56309251 0.66258916 0.78086881]
 [0.62469505 0.64392092 0.6984303  0.78086881 0.88345221]
 [0.78086881 0.79633306 0.84102145 0.91064169 1.        ]]
Kw
[[0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.6984303 ]
 [0.         0.         0.         0.66258916 0.78086881]
 [0.         0.         0.6984303  0.78086881 0.88345221]
 [0.         0.79633306 0.84102145 0.91064169 1.        ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]
